# Machine Learning Project - Pawpularity ##
### Armando Fortes (2021403383), David Pissarra (2021403381)

#### Imports ####

In [ ]:
from tensorflow.keras.metrics import RootMeanSquaredError
from tensorflow.keras.losses import MeanSquaredError
from sklearn.model_selection import train_test_split
from tensorflow.keras.optimizers import Adam
from tensorflow.keras import layers, models
from matplotlib import pyplot as plt
import tensorflow as tf
import pandas as pd
import numpy as np

In [ ]:
physical_devices = tf.config.list_physical_devices('GPU')
print(f'Number of GPUs available: {len(physical_devices)}')
if physical_devices:
    tf.config.experimental.set_memory_growth(physical_devices[0], True)

#### Constants and Hyperparameters ####

In [ ]:
dataset_dir = '../Dataset/'
train_images_path = dataset_dir + 'train/'
test_images_path = dataset_dir + 'test/'
train_meta_path = dataset_dir + 'train.csv'
test_meta_path = dataset_dir + 'test.csv'

In [ ]:
EPOCHS = 10
STRAT_SIZE = 20
BATCH_SIZE = 64
IMAGE_DIM = 128
BUFFER_SIZE = 1024
LEARNING_RATE = 0.001

AUTOTUNE = tf.data.experimental.AUTOTUNE

#### Image Dataset Preprocessing ####

In [ ]:
def load_image(image_path):
    image = tf.io.read_file(image_path)
    image = tf.image.decode_jpeg(image, channels=3)
    image = tf.cast(image, tf.float32) / 255.
    image = tf.image.resize(image, (IMAGE_DIM, IMAGE_DIM))
    return image

In [ ]:
def map_image(image_path, label):
    return load_image(image_path), label

In [ ]:
def attr_distribution(df, attr, value, color="dodgerblue"):
    plt.title(f"{attr} = {value}")
    x = df.loc[df[attr] == value]['Pawpularity']
    x.plot(kind='hist', bins=20, color=color)

    _, max_ylim = plt.ylim()
    plt.axvline(x.mean(), color='k', linestyle='dashed', linewidth=1)
    plt.text(x.mean()*1.1, max_ylim*0.9, 'Mean: {:.2f}'.format(x.mean()))

Training and Validation Sets

In [ ]:
train_metadata = pd.read_csv(train_meta_path).sort_values(by='Id')
train_metadata = train_metadata.assign(Strat=(train_metadata['Pawpularity']//STRAT_SIZE))
images_names = (train_images_path + train_metadata['Id'] + '.jpg').values
images_paws = (train_metadata['Pawpularity']).values
images_strats = (train_metadata['Strat']).values

In [ ]:
fig = plt.figure(figsize=(13, 13))
columns = 4
rows = 6
set = 1

for attr in train_metadata.columns:
    if attr not in ('Id', 'Pawpularity', 'Strat'):
        fig.add_subplot(rows, columns, set)
        attr_distribution(train_metadata, attr, 0, color='orange')
        fig.add_subplot(rows, columns, set + 1)
        attr_distribution(train_metadata, attr, 1)
        set += 2

plt.tight_layout()

In [ ]:
train_images_names, valid_images_names, Y_train, Y_valid = train_test_split(
    images_names,
    images_strats,
    test_size=0.1
    )

In [ ]:
ds_train = tf.data.Dataset.from_tensor_slices((train_images_names, Y_train))
ds_train = ds_train.map(map_image, num_parallel_calls=AUTOTUNE)
# ds_train = ds_train.repeat()
ds_train = ds_train.shuffle(buffer_size=BUFFER_SIZE, reshuffle_each_iteration=True)
ds_train = ds_train.batch(BATCH_SIZE)
ds_train = ds_train.prefetch(AUTOTUNE)

# for x in ds_test:
#     plt.imshow(x[0].numpy())
#     plt.show()
#     break

In [ ]:
ds_valid = tf.data.Dataset.from_tensor_slices((valid_images_names, Y_valid))
ds_valid = ds_valid.map(map_image, num_parallel_calls=AUTOTUNE)
ds_valid = ds_valid.batch(1)
ds_valid = ds_valid.prefetch(AUTOTUNE)

Test Set

In [ ]:
test_metadata = pd.read_csv(test_meta_path).sort_values(by='Id')
test_images_names = (test_images_path + test_metadata['Id'] + '.jpg').values

In [ ]:
ds_test = tf.data.Dataset.from_tensor_slices((test_images_names,))
ds_test = ds_test.map(load_image, num_parallel_calls=AUTOTUNE)
ds_test = ds_test.batch(BATCH_SIZE)
ds_test = ds_test.prefetch(AUTOTUNE)

# for x in ds_test:
#     plt.imshow(x[0].numpy())
#     plt.show()
#     break

#### Model and training ####

In [ ]:
model = models.Sequential()
model.add(layers.Conv2D(64, (3, 3), activation='relu', input_shape=(IMAGE_DIM, IMAGE_DIM, 3)))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(128, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(128, (3, 3), activation='relu'))

model.summary()

In [ ]:
model.add(layers.Flatten())
model.add(layers.Dense(128, activation='relu'))
model.add(layers.Dropout(0.2))
model.add(layers.Dense(32, activation='relu'))
model.add(layers.Dropout(0.2))
model.add(layers.Dense(100//STRAT_SIZE, activation='softmax'))

model.summary()

In [ ]:
model.compile(optimizer=Adam(),
              loss=MeanSquaredError(),
              metrics=['accuracy'])

history = model.fit(ds_train, epochs=EPOCHS, validation_data=ds_valid)

#### Submission ####

In [ ]:
yhat_test = model.predict(ds_valid, verbose=1)

In [ ]:
print(yhat_test)

In [ ]:
test_predictions = pd.DataFrame()
test_predictions['Id'] = test_metadata.Id
test_predictions['Pawpularity'] = yhat_test
test_predictions.to_csv('submission.csv', index=False)

In [ ]:
test_predictions.head(8)